# HW 3 Forward/Inverse Kinematics

For this assignment, we will be working with the 3R(evolute) robotic system (see below)
<img src="./3R-robot.png" width="420">

Note that the angles are measured *relative* to the previous link. This assignment will have you write down the forward/inverse kinematics of the 3R robot using the geometric approach and the Lie group approach. You will use both approaches to create an inverse kinematics solver.

### NOTE: YOU SHOULD KEEP NOTE OF THE CHANGE OF X-Y PLANE ON THE GRAPH TO THE Y-Z PLANE WITH ROTATIONS ABOUT THE X AXIS TO COINCIDE WITH MESHCAT VISUALIZATION

In [1]:
# Import packages 
try: 
    from jax import config
    config.update("jax_enable_x64", True)
    import meshcat
    import meshcat.geometry as geom
    import meshcat.transformations as tf
    import numpy as np
    import time
    import jax.numpy as np
    import jax
    from jax import jacfwd
    from jaxlie import SE2, SE3
    import matplotlib.pyplot as plt
    
    print('Import packages works! Great work following directions :D !')
except Exception as e:
    print('Something went wrong. The following error tells you what happened. Go through README.md again and see what went wrong')
    print(e)

## Helper functions 
def skew3(twist):
    # note the convention is ang vel then linear vel
    # xi = [w,v]
    w = twist[:3]
    v = twist[3:]
    return np.array([[0, -w[2], w[1], v[0]],
                     [w[2], 0, -w[0], v[1]],
                     [-w[1], w[0], 0, v[2]],
                     [0., 0., 0., 0.]])
def skew2(twist):
    # note the convention is ang vel then linear vel
    # xi = [w,v]
    w = twist[0]
    v = twist[1:]
    return np.array([[0, -w, v[0]],
                     [w, 0, v[1]],
                     [0., 0., 0.]])

## Constants
_L1 = 1.2
_L2 = 1.0
_L3 = 0.5

Import packages works! Great work following directions :D !


## Robot Renderer!
Here is a function that renders the robot given joint angles using meshcat! Feel free to use whatever geometry that illustrates the robot's joints and end-effector position (e.g., a sphere) and edit as you see fit.

In [2]:
# Create a jupyter cell that renders the visalizer by calling the function below
# Create a new visualizer
vis = meshcat.Visualizer()
vis.delete()
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7008/static/


Below is a function that takes in the visualizer object and returns a list of joint rotation frames (use as a reference). 
The `render` function that takes in the robot joint list and a list of rotation $\theta$. Reminder that the above diagram is along the y-z plane. You will work as if in the x-y plane. 

In [3]:
def create_robot(vis):
    base = vis['base']
    jnt1 = base['jnt1']
    jnt1.set_object(geom.Sphere(0.1))
    T_jnt12 = jnt1['jnt1_loc']
    T_jnt12.set_transform(tf.translation_matrix(np.array([0., _L1, 0.])))

    jnt2 = T_jnt12['jnt2']
    jnt2.set_object(geom.Sphere(0.1))
    T_jnt23 = jnt2['jnt3_loc']
    T_jnt23.set_transform(tf.translation_matrix(np.array([0., _L2, 0.])))

    jnt3 = T_jnt23['jnt3']
    jnt3.set_object(geom.Sphere(0.1))
    T_jnt3E = jnt3['ee_loc']
    T_jnt3E.set_transform(tf.translation_matrix(np.array([0., _L3, 0.])))
    ee = T_jnt3E['ee']
    ee.set_object(geom.Sphere(0.1))

    return [jnt1, jnt2, jnt3]

robot = create_robot(vis)

def render(robot, th):
    for jnt, _th in zip(robot, th):
        jnt.set_transform(tf.rotation_matrix(_th, [1.,0.,0.]))

render(robot, [0.2,0.3,0.4])



No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


## Q1 Forward Kinematics 
Write out the forward kinematics $p = f(\theta)$, where $p=[x,y,\psi]^\top$ and $\theta \in \mathbb{R}^3$

In [4]:
def fk_analytical(th):
    # input: th is an array of 3 vaues th = [th1 th2 th3]
    # output: x,y,psi as an array
    x = _L1 * np.cos(th[0]) + _L2 * np.cos(th[0] + th[1]) + _L3 * np.cos(th[0] + th[1] + th[2])
    y = _L1 * np.sin(th[0]) + _L2 * np.sin(th[0] + th[1]) + _L3 * np.sin(th[0] + th[1] + th[2])
    psi = th[0] + th[1] + th[2]
    return np.array([x, y, psi])

### Evaluate your answer 
For $\theta = [\pi/2,-\pi/3,\pi/4]^\top$, you should get $p=[0.99543493, 2.18296291, 1.30899694]^\top$. Check your answer below.

In [5]:
th = np.array([np.pi/2,-np.pi/3,np.pi/4])
fk_analytical(th)

Array([0.99543493, 2.18296291, 1.30899694], dtype=float64)

### render your answer
Use the rendering function to visualize your robot. Confirm the results make visual sense.

In [6]:
render(robot, th)
vis.jupyter_cell()

## Q2 Inverse Kinematics 
Using jax's build in Jacobian function `jacfwd`, we will create the function `jac_fk_analytical` which takes as input $\theta$ and outputs the Jacobian matrix.

In [7]:
# here jacfwd is a function from jax that takes in a **function** and returns a **function** that evalutes the derivative at the first argument
jac_fk_analytical = jacfwd(fk_analytical)

### Evaluate your answer 
For the same input for $\theta$ as above, and $\dot{\theta} = [1,1,1]^\top$ you should get the following $J(\theta) \dot{\theta} = v = [-3.64888874, 2.12027938, 3.]^\top$


In [8]:
th = np.array([np.pi/2,-np.pi/3,np.pi/4])
thdot = np.array([1.,1.,1.])
jac_fk_analytical(th)@thdot

Array([-3.64888874,  2.12027938,  3.        ], dtype=float64)

## Q3 Inverse Kinematics Solver 
Using Newton's method, create a function that takes a Newton's step towards the target position $p_d = [1.5,1.5, 0.]^\top$. 

Start with creating the residual function `res` that takes as input $p_d$ and $\theta$ and returns $p_d - f(\theta)$ where $f(\theta)$ is the forward kinematics. That is create the function $r(p_d, \theta) = p_d - f(\theta)$

In [9]:
def res(pd, th):
    # input is desired pose as x,y,psi, and robot joint positions
    # output return is a vector representing pd - f(\theta)
    return pd - fk_analytical(th)

Using the residual function, write a function `ik_newton_step` that takes in $p_d$ and $\theta_k$ -- an estimate of $\theta$ -- and returns $\theta_{k+1}$ -- an update based on the iteration $\theta_{k+1} = \theta_k + J(\theta)^\dagger r(p_d, \theta)$

In [10]:
def ik_newton_step(pd, th):
    # input is desired pose as x,y,psi, and robot joint positions at iteration k
    # output return is a vector representing \theta_{k+1}
    return th + np.linalg.pinv(jac_fk_analytical(th)) @ res(pd, th)

## Q4 Animate Solver Iterations

Write a function that iterates through the Newton's step to solve for the inverse kinematic solution where the desired end-effector pose is $p_d = [1.5, 1.5, 0.]^\top$ with initial guess $\theta_0 = [\pi/2, -\pi/3, \pi/4]^\top$. Iterate a total of 10 times with a pause of $0.5 s$ using the `time.sleep` function.

Create a reference frame for the target location $p_d$ below. 

In [11]:
render(robot, th)
target_loc = vis['p_d']
target_loc.set_transform(tf.translation_matrix(np.array([0., 1.5, 1.5])))
target_loc.set_object(geom.triad())
vis.jupyter_cell()

### Write the animation code below

In [12]:
pd = np.array([1.5,1.5,0.])
th = np.array([np.pi/2,-np.pi/3,np.pi/4])
for _ in range(10):
    ##############
    # FILL IN CODE HERE WITH NEWTON STEP
    ###############
    th = ik_newton_step(pd, th)
    render(robot, th)
    time.sleep(0.5)

print('Solution : ')
print(th, fk_analytical(th), res(pd, th))

Solution : 
[ 1.53216773 -1.22653653 -0.30563121] [1.50000000e+00 1.50000000e+00 5.55111512e-17] [ 0.00000000e+00  0.00000000e+00 -5.55111512e-17]


## Q4 Forward Kinematics via Lie Groups
Write the forward kinematics using SE(2) groups with jaxlie `SE2` package. Assume xy planar rotations are in the yz plane for simplicity with visualization. Compare results with the analytical expression to confirm.

In [13]:
def fk_lie(th):
    # input is theta
    # output is SE2 transform describing the transform from frame A to G: T_AG
    # use the SE2.from_xy_theta(x,y,th) function to create an SE2 transform
    gAB = SE2.from_xy_theta(0, 0, th[0])
    gBC = SE2.from_xy_theta(_L1, 0, 0)
    gCD = SE2.from_xy_theta(0, 0, th[1])
    gDE = SE2.from_xy_theta(_L2, 0, 0)
    gEF = SE2.from_xy_theta(0, 0, th[2])
    gFG = SE2.from_xy_theta(_L3, 0, 0)
    T_AG = gAB @ gBC @ gCD @ gDE @ gEF @ gFG
    return T_AG

In [14]:
# check if results are correct here
th = np.array([np.pi/2,-np.pi/3,np.pi/4])
print(
    'SE(2)', fk_lie(th).translation(), fk_lie(th).rotation().as_radians(), '\n'
    'From Geometry', fk_analytical(th)
)

SE(2) [0.99543493 2.18296291] 1.3089969389957472 
From Geometry [0.99543493 2.18296291 1.30899694]


## Q 5 Inverse Kinematics via Lie Groups

First, write down the expression for the body-velocity. This first requires that we evaluate the adjoint maps for each joint from the joint location to the end-effector frame. Since each rotation occurs about the rotational axis, use unit vectors to pull out the resulting Jacobian. You can write the expression out as 
$$
    \begin{align}
        \hat{\xi} &= T_{AG}^{-1} \left( 
            T_{AB} \hat{\omega}_1 T_{BG} \Delta \theta_1 + T_{AD} \hat{\omega}_2 T_{DG} \Delta \theta_2 + T_{AF} \hat{\omega}_3 T_{FG} \Delta \theta_3
        \right) \\ 
        &= \textbf{Ad}_{T_{GB}} \omega_1 \Delta \theta_1 + \textbf{Ad}_{T_{GD}} \omega_2 \Delta \theta_2 + \textbf{Ad}_{T_{GF}} \omega_2 \Delta \theta_2 \\ 
        &= \begin{bmatrix}
            \textbf{Ad}_{T_{GB}} \omega_1  &  \textbf{Ad}_{T_{GD}} \omega_2 & \textbf{Ad}_{T_{GF}} \omega_2
            \end{bmatrix} 
            \begin{bmatrix}
                \Delta \theta_1 \\ \Delta \theta_2 \\ \Delta \theta_3
            \end{bmatrix} \\ 
        &= J(\theta) \Delta \theta
    \end{align}
$$
where $T_{AG}$ is the transform from frame A to G(end-effector) and $\omega_i$ are the unit vectors of the joint degrees of freedom. 

In [15]:
def jac_lie(th):
    # input is the joint angles of the robot
    # output is the Jacobian matrix as described above
    gAB = SE2.from_xy_theta(0, 0, th[0])
    gBC = SE2.from_xy_theta(_L1, 0, 0)
    gCD = SE2.from_xy_theta(0, 0, th[1])
    gDE = SE2.from_xy_theta(_L2, 0, 0)
    gEF = SE2.from_xy_theta(0, 0, th[2])
    gFG = SE2.from_xy_theta(_L3, 0, 0)
    T_GB = gBC @ gCD @ gDE @ gEF @ gFG
    T_GD = gDE @ gEF @ gFG
    T_GE = gEF @ gFG
    return np.vstack(((T_GB.inverse().adjoint() @ np.array([0, 0, 1])), (T_GD.inverse().adjoint() @ np.array([0, 0, 1])), (T_GE.inverse().adjoint() @ np.array([0, 0, 1])))).T


### Evaluate your answer 
For the same input for $\theta$ as above, and $\dot{\theta} = [1,1,1]^\top$ you should get the following $J(\theta) \dot{\theta} = \xi = [1.10363071, 4.07332455, 3.        ]^\top$

In [16]:
th = np.array([np.pi/2,-np.pi/3,np.pi/4])
thdot = np.array([1.,1.,1.])
jac_lie(th) @ thdot

Array([1.10363071, 4.07332455, 3.        ], dtype=float64)

## Q 6 Inverse Kinematics Solver Lie Group
Let us now write down the residual function on $SE(2)$ that takes in a desired transform $T_d$ and returns $\log(T_d^{-1}T_{AG}(\theta))$

In [17]:
def res(Td, th):
    # input is a SE2 transform and the joint position
    # output is a 3dim vector representing the relative twist between two SE2 elements
    # hint: use fk_lie function that you wrote before and use the .log() function of the result interior matrix multiplication
    return (Td.inverse() @ fk_lie(th)).log()

Evaluate your answer. For the desired location specified above, you should get as a result is $r(T_d, \theta) = [0.04536114, 1.25082403, 1.41371669]^\top$

In [18]:
pd = np.array([1.5,1.5,0.])
Td = SE2.from_xy_theta(pd[0], pd[1], pd[2])
th = np.array([np.pi/2,-np.pi/4,np.pi/5])

res(Td, th)

Array([0.04536114, 1.25082403, 1.41371669], dtype=float64)

Using the residual function, write a function `ik_lie_newton_step` that takes in $T_d$ and $\theta_k$ -- an estimate of $\theta$ -- and returns $\theta_{k+1}$ -- an update based on the iteration $\theta_{k+1} = \theta_k - J(\theta)^\dagger r(p_d, \theta)$

NOTE THAT THE UPDATE EQUATION HAS A MINUS, RATHER THAN A PLUS. 
This is due to the Jacobian being defined over the tangent space rather than with the Euclidean distance and the formulation varying slightly. The meaning still stays the same. 

In [19]:
def ik_lie_newton_step(Td, th):
    # input is target transform in SE2 and joint angles at iteration k
    # output is a Newton step result for IK \theta_{k+1}
    return th - np.linalg.pinv(jac_lie(th)) @ res(Td, th)

## Q7 Animate Solver Iterations

Write a function that iterates through the Newton's step to solve for the inverse kinematic solution where the desired end-effector pose is $T_d(p_d=[1.5, 1.5], \phi_d=0.)$ with initial guess $\theta_0 = [\pi/2, -\pi/3, \pi/4]^\top$. Iterate a total of 10 times with a pause of $0.5 s$ using the `time.sleep` function.

Create a reference frame for the target location $T_d$ below. 

In [20]:
render(robot, th)
target_loc = vis['T_d']
target_loc.set_transform(tf.translation_matrix(np.array([0., 1.5, 1.5])))
target_loc.set_object(geom.triad())
vis.jupyter_cell()

### Write the animation code below

In [23]:
pd = np.array([1.5,1.5,0.])
Td = SE2.from_xy_theta(pd[0], pd[1], pd[2])
th = np.array([np.pi/2,-np.pi/4,np.pi/5])

for _ in range(10):
    ##########
    # Add you newton step code here
    ##########
    th = ik_lie_newton_step(Td, th)
    render(robot, th)
    time.sleep(0.5)

print('Solution : ')
print(th, fk_lie(th), res(Td, th))

Solution : 
[ 1.53216773 -1.22653653 -0.30563121] SE2(unit_complex=[1. 0.], xy=[1.5 1.5]) [0.0000000e+00 0.0000000e+00 1.1871249e-18]
